# Chatbot
In this notebook, we'll build a chatbot.  The chatbot interface will use gradio.  Underlying the chatbot is the Neo4j knowledge graph we built in previous labs.  The chatbot uses generative AI and langchain.  We take a natural language question, convert it to Neo4j Cypher using generative AI and run that against the database.  We take the response from the database and use generative AI again to convert that back to natural language before presenting it to the user.

## Base Example Without Grounding
Before grounding with the Neo4j, let's setup up a baseline that just uses an LLM to answer questions.

In [3]:
from langchain_google_vertexai import VertexAI

base_chain = VertexAI(model_name='gemini-1.0-pro-002', max_output_tokens=2048, temperature=0)

We can now ask a simple finance question.

In [4]:
base_response = base_chain.invoke("""What are the top 10 investments for Rempart?""")
print(f"Final answer: {base_response}")

Final answer: ## Top 10 Investments for Rempart

Rempart is a French company specializing in the design, manufacture, and sale of security and defense equipment. The company's products are used by military and law enforcement agencies around the world.

Here are the top 10 investments for Rempart:

**1. Research and development:** Rempart should continue to invest in research and development to stay ahead of the competition and develop new and innovative products. This could include investing in new technologies such as artificial intelligence and robotics.

**2. Marketing and sales:** Rempart should invest in marketing and sales to increase brand awareness and generate new leads. This could include attending trade shows, developing a strong online presence, and hiring experienced sales staff.

**3. Manufacturing:** Rempart should invest in its manufacturing facilities to improve efficiency and quality. This could include investing in new equipment, automating processes, and improving 

While this answer may seem reasonable, we have no real way to know how the LLM came it with it, or where it was sourced from.

Here is a more complicated example where we expect the LLM to understand some more specific terminology.

In [5]:
base_response = base_chain.invoke("""Which managers own FAANG stocks?""")
print(f"Final answer: {base_response}")

Final answer: ## FAANG Stock Ownership by Managers

Here's a breakdown of FAANG stock ownership by managers as of November 2023:

**Meta (Facebook):**

* **Mark Zuckerberg:** Owns 351 million shares, representing 13.4% of the company.
* **Sheryl Sandberg:** Owns 17.5 million shares, representing 0.6% of the company.
* **David Wehner:** Owns 1.5 million shares, representing 0.05% of the company.

**Apple:**

* **Tim Cook:** Owns 847,969 shares, representing 0.02% of the company.
* **Arthur Levinson:** Owns 143,227 shares, representing 0.004% of the company.
* **Luca Maestri:** Owns 125,000 shares, representing 0.003% of the company.

**Amazon:**

* **Jeff Bezos:** Owns 10% of the company.
* **Andy Jassy:** Owns 64,748 shares, representing 0.002% of the company.
* **Brian Olsavsky:** Owns 48,623 shares, representing 0.001% of the company.

**Netflix:**

* **Reed Hastings:** Owns 1.8 million shares, representing 1.9% of the company.
* **Ted Sarandos:** Owns 570,000 shares, representing 0.

In this case, it looks like the LLM understands the ubiquitous acronym FAANG but, unsurprisingly, the results indicate it doesn't understand manager within the context of our data model.  In your use case, you may have lots of specific terminology/ontology like this that you would need a chatbot to understand.

## Grounding LLMs with Neo4j
Now let's create a chatbot that is grounded with Neo4j. Below is the pattern we will follow with LangChain:

![](images/langchain.png)

## Cypher Generation
We have to use a prompt template that: 
1. Clearly states what schema to use 
2. Provides principles the chatbot should follow in generating responses
3. Demonstrates few-shot examples to help the chatbot be more accurate in its query generation.

In [6]:
CYPHER_GENERATION_TEMPLATE = """You are an expert Neo4j Cypher translator who understands the question in english and convert to Cypher strictly based on the Neo4j Schema provided and following the instructions below:
1. Generate Cypher query compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE, CONTAINS ANY keywords in the cypher. Use alias when using the WITH keyword
3. Please do not use same variable names for different nodes and relationships in the query.
4. Use only Nodes and relationships mentioned in the schema
5. Always enclose the Cypher output inside 3 backticks
6. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Company name use `toLower(c.name) contains 'neo4j'`
7. Candidate node is synonymous to Manager
8. Always use aliases to refer the node in the query
9. 'Answer' is NOT a Cypher keyword. Answer should never be used in a query.
10. Please generate only one Cypher query per question. 
11. Cypher is NOT SQL. So, do not mix and match the syntaxes.
12. Every Cypher query always starts with a MATCH keyword.
13. Always use `IN` keyword instead of `CONTAINS ANY`

Schema:
{schema}
Samples:
Question: Which fund manager owns most shares? What is the total portfolio value?
Answer: MATCH (m:Manager) -[o:OWNS]-> (c:Company) RETURN m.managerName as manager, sum(distinct o.shares) as ownedShares, sum(o.value) as portfolioValue ORDER BY ownedShares DESC LIMIT 10

Question: Which fund manager owns most companies? How many shares?
Answer: MATCH (m:Manager) -[o:OWNS]-> (c:Company) RETURN m.managerName as manager, count(distinct c) as ownedCompanies, sum(distinct o.shares) as ownedShares ORDER BY ownedCompanies DESC LIMIT 10

Question: What are the top 10 investments for Vanguard?
Answer: MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(m.managerName) contains "vanguard" RETURN c.companyName as Investment, sum(DISTINCT o.shares) as totalShares, sum(DISTINCT o.value) as investmentValue order by investmentValue desc limit 10

Question: What other fund managers are investing in same companies as Vanguard?
Answer: MATCH (m1:Manager) -[:OWNS]-> (c1:Company) <-[o:OWNS]- (m2:Manager) WHERE toLower(m1.managerName) contains "vanguard" AND elementId(m1) <> elementId(m2) RETURN m2.managerName as manager, sum(DISTINCT o.shares) as investedShares, sum(DISTINCT o.value) as investmentValue ORDER BY investmentValue LIMIT 10

Question: What are the top investors for Apple?
Answer: MATCH (m1:Manager) -[o:OWNS]-> (c1:Company) WHERE toLower(c1.companyName) contains "apple" RETURN distinct m1.managerName as manager, sum(o.value) as totalInvested ORDER BY totalInvested DESC LIMIT 10

Question: Which managers own FAANG stocks?
Answer: MATCH (m:Manager)-[o:OWNS]->(c:Company) WHERE toLower(c.companyName) IN ["facebook", "amazon", "apple", "netflix", "google"] RETURN m.managerName AS manager, c.companyName AS company

Question: What are the other top investments for fund managers investing in Apple?
Answer: MATCH (c1:Company) <-[:OWNS]- (m1:Manager) -[o:OWNS]-> (c2:Company) WHERE toLower(c1.companyName) contains "apple" AND elementId(c1) <> elementId(c2) RETURN DISTINCT c2.companyName as company, sum(DISTINCT o.value) as totalInvested, sum(DISTINCT o.shares) as totalShares ORDER BY totalInvested DESC LIMIT 10

Question: What are the top investors in the last 3 months?
Answer: MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE date() > o.reportCalendarOrQuarter > o.reportCalendarOrQuarter - duration({{months:3}}) RETURN distinct m.managerName as manager, sum(o.value) as totalInvested, sum(o.shares) as totalShares ORDER BY totalInvested DESC LIMIT 10

Question: Which managers own FAANG stocks?
Answer: MATCH (m:Manager)-[o:OWNS]->(c:Company) WHERE toLower(c.companyName) IN ["facebook", "amazon", "apple", "netflix", "google"] RETURN m.managerName AS manager, c.companyName AS company

Question: What are top investments in last 6 months for Vanguard?
Answer: MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(m.managerName) contains "vanguard" AND date() > o.reportCalendarOrQuarter > date() - duration({{months:6}}) RETURN distinct c.companyName as company, sum(o.value) as totalInvested, sum(o.shares) as totalShares ORDER BY totalInvested DESC LIMIT 10

Question: Who are Apple's top investors in last 3 months?
Answer: MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(c.companyName) contains "apple" AND date() > o.reportCalendarOrQuarter > date() - duration({{months:3}}) RETURN distinct m.managerName as investor, sum(o.value) as totalInvested, sum(o.shares) as totalShares ORDER BY totalInvested DESC LIMIT 10

Question: Which fund manager under 200 million has similar investment strategy as Vanguard?
Answer: MATCH (m1:Manager) -[o1:OWNS]-> (:Company) <-[o2:OWNS]- (m2:Manager) WHERE toLower(m1.managerName) CONTAINS "vanguard" AND elementId(m1) <> elementId(m2) WITH distinct m2 AS m2, sum(distinct o2.value) AS totalVal WHERE totalVal < 200000000 RETURN m2.managerName AS manager, totalVal*0.000001 AS totalVal ORDER BY totalVal DESC LIMIT 10

Question: Who are common investors in Apple and Amazon?
Answer: MATCH (c1:Company) <-[:OWNS]- (m:Manager) -[:OWNS]-> (c2:Company) WHERE toLower(c1.companyName) contains "apple" AND toLower(c2.companyName) CONTAINS "amazon" RETURN DISTINCT m.managerName LIMIT 50

Question: What are Vanguard's top investments by shares for 2023?
Answer: MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(m.managerName) CONTAINS "vanguard" AND date({{year:2023}}) = date.truncate('year',o.reportCalendarOrQuarter) RETURN c.companyName AS investment, sum(o.value) AS totalValue ORDER BY totalValue DESC LIMIT 10

Question: What are Vanguard's top investments by value for 2023?
Answer: MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(m.managerName) CONTAINS "vanguard" AND date({{year:2023}}) = date.truncate('year',o.reportCalendarOrQuarter) RETURN c.companyName AS investment, sum(o.shares) AS totalShares ORDER BY totalShares DESC LIMIT 10

Question: {question}
Answer: 
"""

Now let’s create a LangChain prompt template.  

This template defines the parameter inputs for the prompt sent to the Cypher generation bot.  In our example, the inputs will be schema and question.  The question comes from the end user.  The LangChain GraphCypherQAChain automatically inserts the schema via a built-in method to Neo4jGraph.

In [7]:
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=['schema','question'], validate_template=True, template=CYPHER_GENERATION_TEMPLATE
)

Provide your Neo4j credentials.  We need the DB conection URL and password.  The username is neo4j by default.

In [8]:
# username is neo4j by default
NEO4J_USERNAME = 'neo4j'

# You will need to change these to match your credentials
NEO4J_URI = 'neo4j+s://7e098dc5.databases.neo4j.io' #Eg 'neo4j+s://ccc5f4f5.databases.neo4j.io'
NEO4J_PASSWORD = '19_Kt_bHLte_0F2r4OAqGv1ihz9ciXJfJaav0-hpNrI'

We need to connect to the graph via LangChain.

In [9]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD
)

In [14]:
graph.schema

'Node properties:\nManager {managerName: STRING}\nCompany {companyName: STRING, cusip: STRING}\nDocument {documentId: STRING, documentType: STRING, seqId: INTEGER, textEmbedding: LIST, text: STRING}\nRelationship properties:\nOWNS {reportCalendarOrQuarter: DATE, shares: INTEGER, value: FLOAT}\nSIMILAR {score: FLOAT}\nThe relationships:\n(:Manager)-[:OWNS]->(:Company)\n(:Manager)-[:SIMILAR]->(:Manager)'

We define our `chain` object (specifically a`GraphCypherQAChain`) using two vertex AI LLMs:
* [gemini-1.5-flash](https://ai.google.dev/gemini-api/docs/models/gemini#gemini-1.5-flash) to translate user questions to Cypher queries
* [gemini-1.5-pro](https://ai.google.dev/gemini-api/docs/models/gemini#gemini-1.5-pro)  to convert Cypher query results back to natural language for human-friendly responses. 

`GraphCypherQAChain` also takes a ‘Neo4jGraph’ so it can handle the chatbot process end-to-end, from taking the user question and translating to Cypher to executing the query, getting results, translating back to natural language, and returning to the user. 


In [10]:
from langchain.chains import GraphCypherQAChain
import json

chain = GraphCypherQAChain.from_llm(
    VertexAI(model_name='gemini-1.5-flash-001', max_output_tokens=2048, temperature=0.0),
    graph=graph,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    verbose=True,
    return_direct=True
)

def chat(que):
    r = chain.invoke(que)
    print(r)
    llm=VertexAI(model_name='gemini-1.5-pro-001', max_output_tokens=2048, temperature=0.0)
    summary_prompt_tpl = f"""Human: 
    Fact: {json.dumps(r['result'])}

    * Summarise the above fact as if you are answering this question "{r['query']}"
    * When the result inside fact is not empty, assume the question is valid and the answer is true
    * Do not return helpful or extra text or apologies
    * Just return summary to the user. DO NOT start with Here is a summary
    * List the result in rich text format if there are more than one results
    * If there is an empty result inside Facts, do not try to provide your own answer
    Assistant:
    """
    return llm.invoke(summary_prompt_tpl)

Below we have a few examples of how we can get answers from the chatbot.

## Why Ground Your LLM?
Recall our base example where we asked for the top 10 Rempart investments?  We got an answer that looked like it may be reasonable, but we couldn't validate it or track sources.  We also asked what managers own FAANG stocks, and for that, we unsurprisingly received the wrong answers for our use case.

Let's try again grounding with Neo4j. 

In [10]:
r2 = chat("""What are the top 10 investments for Rempart?""")
print(f"Final answer: {r2}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Manager)-[o:OWNS]->(c:Company)
WHERE toLower(m.managerName) CONTAINS "rempart"
RETURN c.companyName AS investment, sum(o.value) AS totalValue ORDER BY totalValue DESC LIMIT 10


> Finished chain.
{'query': 'What are the top 10 investments for Rempart?', 'result': [{'investment': 'Microsoft Corp', 'totalValue': 133620}, {'investment': 'THERMO FISHER SCIENTIFIC', 'totalValue': 130348}, {'investment': 'Costco Wholesale ', 'totalValue': 125544}, {'investment': 'LOWES', 'totalValue': 96688}, {'investment': 'CANADIAN NATIONAL RAILWAY', 'totalValue': 94160}, {'investment': 'BANK OF MONTREAL', 'totalValue': 92496}, {'investment': 'ROPER TECHNOLOGIES', 'totalValue': 89776}, {'investment': 'Johnson & Johnson', 'totalValue': 86360}, {'investment': 'COLGATE PALMOLIVE', 'totalValue': 78988}, {'investment': 'JPMorgan Chase & Co', 'totalValue': 75740}]}
Final answer: The top 10 investments for Rempart are:
* **Microsoft Co

Notice that this answer is different from our base example, and this time we have the Cypher logic used to obtain the answer from our database. This means that we can trace back how we came up with this answer and make any adjustments to our database or prompt if we need to.

Now lets try the FAANG question.

In [11]:
r3 = chat("""Which managers own FAANG stocks?""")
print(f"Final answer: {r3}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Manager)-[o:OWNS]->(c:Company) WHERE toLower(c.companyName) IN ["facebook", "amazon", "apple", "netflix", "google"] RETURN m.managerName AS manager, c.companyName AS company


> Finished chain.
{'query': 'Which managers own FAANG stocks?', 'result': [{'manager': 'Beacon Wealthcare LLC', 'company': 'Apple'}, {'manager': 'Pinnacle Holdings, LLC', 'company': 'Google'}, {'manager': 'Pinnacle Holdings, LLC', 'company': 'Google'}]}
Final answer: **Beacon Wealthcare LLC** owns Apple stock. <br>
**Pinnacle Holdings, LLC** owns Google stock. 



Here again, we notice the traceability with Cypher, and because we engineered our prompt to include our schema, it understood what “manager” means in the context of our use case.

## Why Ground your LLM with Neo4j?
There are 3 primary reasons to ground your LLM with Neo4j specifically:
1. __Grounding for more complex question handling__: Multi-hop knowledge retrieval across connected data. Connections between data points are calculated before query time. 
2. __Enterprise reliability and security__: Fine-grained security so the chatbot only accesses information the user has permission to. Autonomous clustering for horizontal scaling.  Fully managed service in the cloud through Aura. 
3. __Performance__: fast queries with high concurrency for many users.

We can explore point 1 with more complex questions below.

A question requiring ~4 hops (would be joins in the relational world).  Having a knowledge graph with relationships calculated before query time allows us to answer the question quickly.

In [12]:
r4 = chat("""What are the other top investments for fund managers investing in Lilly?""")
print(f"Final answer: {r4}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c1:Company) <-[:OWNS]- (m1:Manager) -[o:OWNS]-> (c2:Company) WHERE toLower(c1.companyName) CONTAINS "lilly" AND elementId(c1) <> elementId(c2) RETURN DISTINCT c2.companyName as company, sum(DISTINCT o.value) as totalInvested, sum(DISTINCT o.shares) as totalShares ORDER BY totalInvested DESC LIMIT 10


> Finished chain.
{'query': 'What are the other top investments for fund managers investing in Lilly?', 'result': [{'company': 'Microsoft Corp', 'totalInvested': 2290947843977000.0, 'totalShares': 17393776522}, {'company': 'Apple Inc', 'totalInvested': 2230638696081000.0, 'totalShares': 28033302819}, {'company': 'Amazon Inc', 'totalInvested': 969104304752000.0, 'totalShares': 18767636504}, {'company': 'UnitedHealth Group', 'totalInvested': 672149591363000.0, 'totalShares': 3195110514}, {'company': 'NVIDIA Corp', 'totalInvested': 611047372090000.0, 'totalShares': 5833290176}, {'company': 'Johnson & Johnson', 'tota

Combine also with property conditions.

In [13]:
r5 = chat("""Which fund managers under 200 million have the most similar investment strategies to Rempart? Return the top 10.""")
print(f"Final answer: {r5}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m1:Manager)-[o1:OWNS]->(:Company)<-[o2:OWNS]-(m2:Manager)
WHERE toLower(m1.managerName) CONTAINS "rempart" AND elementId(m1) <> elementId(m2)
WITH distinct m2 AS m2, sum(distinct o2.value) AS totalVal
WHERE totalVal < 200000000
RETURN m2.managerName AS manager, totalVal*0.000001 AS totalVal
ORDER BY totalVal DESC
LIMIT 10


> Finished chain.
{'query': 'Which fund managers under 200 million have the most similar investment strategies to Rempart? Return the top 10.', 'result': [{'manager': 'BARD ASSOCIATES INC', 'totalVal': 197.88}, {'manager': 'Columbia Asset Management', 'totalVal': 195.07899999999998}, {'manager': 'Noked Israel Ltd', 'totalVal': 193.083}, {'manager': 'Meiji Yasuda Life Insurance Co', 'totalVal': 191.873}, {'manager': 'Doyle Wealth Management', 'totalVal': 189.45999999999998}, {'manager': 'Triple Frond Partners LLC', 'totalVal': 187.261}, {'manager': 'D L CARLSON INVESTMENT GROUP INC', 'totalV

and more...

In [14]:
r6 = chat("""Please get me common investors between Tesla and Costco""")
print(f"Final answer: {r6}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (c1:Company)<-[:OWNS]-(m:Manager)-[:OWNS]->(c2:Company) WHERE toLower(c1.companyName) CONTAINS "tesla" AND toLower(c2.companyName) CONTAINS "costco" RETURN DISTINCT m.managerName


> Finished chain.
{'query': 'Please get me common investors between Tesla and Costco', 'result': [{'m.managerName': '180 WEALTH ADVISORS, LLC'}, {'m.managerName': '1832 Asset Management L.P.'}, {'m.managerName': '1834 INVESTMENT ADVISORS CO'}, {'m.managerName': '1900 WEALTH MANAGEMENT LLC'}, {'m.managerName': '25 LLC'}, {'m.managerName': '360 Financial, Inc.'}, {'m.managerName': '4J Wealth Management LLC'}, {'m.managerName': '626 Financial, LLC'}, {'m.managerName': 'ABEILLE ASSET MANAGEMENT SA'}, {'m.managerName': 'ABLE Financial Group, LLC'}]}
Final answer: **180 WEALTH ADVISORS, LLC**, **1832 Asset Management L.P.**, **1834 INVESTMENT ADVISORS CO**, **1900 WEALTH MANAGEMENT LLC**, **25 LLC**, **360 Financial, Inc.**, **4J Wealth Manageme

## Grounded Chatbot
Now we will use Gradio to deploy a chat interface with our chain behind it.

The below code deploys a Gradio application.  You can access the app via a local URL. A publicly sharable URL is also provided (sharable for 3 days).

In [15]:
import gradio as gr
import typing_extensions
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key = "chat_history", return_messages = True)

def chat_response(input_text,history):

    try:
        return chat(input_text)
    except:
        # a bit of protection against exposed error messages
        # we could log these situations in the backend to revisit later in development
        return "I'm sorry, there was an error retrieving the information you requested."

interface = gr.ChatInterface(fn = chat_response,
                             title = "Investment Chatbot",
                             description = "powered by Neo4j",
                             theme = "soft",
                             chatbot = gr.Chatbot(height=500),
                             undo_btn = None,
                             clear_btn = "\U0001F5D1 Clear chat",
                             examples = ["What are the top 10 investments for Rempart?",
                                         "Which manager owns FAANG stocks?",
                                         "What are other top investments for fund managers investing in Exxon?",
                                         "What are Rempart's top investments by value for 2021?",
                                         "Who are the common investors between Tesla and Costco?",
                                         "Who are Tesla's top investors in last 48 months?"])

interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://368aa84dd81062513b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c1:Company)<-[:OWNS]-(m1:Manager)-[o:OWNS]->(c2:Company)
WHERE toLower(c1.companyName) CONTAINS "exxon" AND elementId(c1) <> elementId(c2)
RETURN DISTINCT c2.companyName AS company, sum(DISTINCT o.value) AS totalInvested, sum(DISTINCT o.shares) AS totalShares
ORDER BY totalInvested DESC
LIMIT 10


> Finished chain.
{'query': 'What are other top investments for fund managers investing in Exxon?', 'result': [{'company': 'Microsoft Corp', 'totalInvested': 2283255196882136.0, 'totalShares': 17090329639}, {'company': 'Apple Inc', 'totalInvested': 2228735980632931.0, 'totalShares': 27952286708}, {'company': 'Amazon Inc', 'totalInvested': 969254135830842.0, 'totalShares': 18762490387}, {'company': 'UnitedHealth Group', 'totalInvested': 671827298215494.0, 'totalShares': 3127915241}, {'company': 'NVIDIA Corp', 'totalInvested': 605486707369299.0, 'totalShares': 5781913891}, {'company': 'Johnson & Johnson', 'totalInveste

## Semantic Layers
The Text to Cypher approach we just did above can be brittle due to the indeterministic nature of models. For Production, you might need to be more deterministic and robust. [Semantic Layers approach](https://github.com/neo4j-partners/neo4j-generative-ai-google-cloud/blob/main/assetmanager/ui/streamlit/semantic_layer/semantic_fn.py) is more suitable here. It leverages on Tooling or Function-calling capabilities of LLMs. So, user queries can be directed to relevant templated cypher queries via function-calling. This way, no Cypher gets geenrated by the LLM. Instead, user's intent and query parameters are captured and directed to the relevant templated cypher code.

## Fine Tuning for Cypher Generation
We encourage you to use Vertex AI Gemini family models with a schema, few-shot examples, and precise prompt engineering for Cypher generation. However, if that still doesn't provide an appropriate level of quality, or you need your LLM to improve accuracy on a more specific task area, you can try fine-tuning.

Fine-tuning is the process of taking a foundational model and making precise changes to improve its performance for a specific, narrower task. It works by taking in training data containing many examples of your specific task and using it to update or add additional parameters in a new version of the model.

The total training time generally takes more than an hour. The tuned adapter model is going to stay within your tenant, and your training data will not be used to train the base model, which is frozen. Tuning runs on GCP's TPU infrastructure that is optimized to run ML workloads.

The training data should be structured as a supervised training set, where each row contains input text and desired, resulting, Cypher query. Vertex AI expects you to adhere to the below format in a `jsonl` file.

```
{"input_text": "MY_INPUT_PROMPT", "output_text": "CYPHER_QUERY"}
```
You can find more about fine-tuning in the [Vertex AI documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models)


## Conclusion
In this notebook, we went through the steps of connecting a LangChain agent to a Neo4j database and using it to generate Cypher queries in response to user requests via LLMs on Vertex AI, thus grounding the LLM with a knowledge graph.

While we used the `gemini-1.5-flash` and `gemini-1.5-pro` models here, this approach can be generalized to other Vertex AI LLMs.  This process can also be augmented with additional steps around the generation chain to customize the experience for specific use cases.  

The critical takeaway is the importance of Neo4j Knowledge Graph as a grounding database to: 
* Anchor your chatbot to reality as it generates responses and 
* Enable your LLM to provide answers enriched with relevant enterprise data.